Syntax natürlicher Sprachen, WS 2023/24

# 02 - Aufgabenblatt (Lösung)

In [1]:
import nltk
from nltk.tree import Tree


## Aufgabe 1 - Ergänzung um Phrasenkategorien


#### Ergänzen Sie in folgendem Klammerausdruck (aus dem vorherigen Aufgabenblatt) die Phrasenkategorie-Label und generieren Sie den entsprechenden Syntaxbaum, indem Sie die Codezelle anschließend ausführen.

In [2]:
#Lösung
tree = Tree.fromstring("""
(S
    (NP
        (NP Die_Kunst)
        (NP des_Ausruhens)
    )
    (VP
        ist
        (NP
            (NP ein_Teil)
            (NP
                (NP der_Kunst)
                (NP des_Arbeitens)
            )
        )
    )
)
""")

tree.pretty_print(unicodelines=True)

                             S                                          
           ┌─────────────────┴────────────┐                              
           │                              VP                            
           │                 ┌────────────┴──────┐                       
           │                 │                   NP                     
           │                 │     ┌─────────────┴──────┐                
           NP                │     │                    NP              
    ┌──────┴────────┐        │     │             ┌──────┴────────┐       
    NP              NP       │     NP            NP              NP     
    │               │        │     │             │               │       
Die_Kunst     des_Ausruhens ist ein_Teil     der_Kunst     des_Arbeitens




## Aufgabe 2 - Phrasenstrukturbaum

#### Geben Sie für den folgenden Satz einen vollständigen Phrasenstrukturbaum an.

*Die Studierende schenkte ihnen ein Buch von Chomsky.*


In [3]:
#Lösung
tree = Tree.fromstring("""
(S
    (NP (DET Die) (N Studierende))
    (VP
        (V schenkte)
        (NP (PRON ihnen))
        (NP 
            (NP (DET ein) (N Buch))
            (PP (P von) (NP (PROPN Chomsky)))
        )
    )
)
""")

tree.pretty_print(unicodelines=True)

                               S                                   
     ┌─────────────────────────┴────┐                               
     │                              VP                             
     │                 ┌───────┬────┴────────────┐                  
     │                 │       │                 NP                
     │                 │       │        ┌────────┴───────┐          
     │                 │       │        │                PP        
     │                 │       │        │            ┌───┴─────┐    
     NP                │       NP       NP           │         NP  
 ┌───┴───────┐         │       │    ┌───┴───┐        │         │    
DET          N         V      PRON DET      N        P       PROPN 
 │           │         │       │    │       │        │         │    
Die     Studierende schenkte ihnen ein     Buch     von     Chomsky



## Aufgabe 3 - Phrasenstrukturgrammatik

#### Schreiben Sie eine kontextfreie Grammatik, die den (um die PP gekürzten) Satz aus der vorherigen Aufgabe erkennt:

*Die Studierende schenkte ihnen ein Buch.*


In [4]:
sent = "die Studierende schenkte ihnen ein Buch"

In [5]:
#Lösung
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> DET N | PRON
    VP -> V NP NP
    DET -> "die" | "ein"
    N -> "Studierende" | "Buch"
    PRON -> "ihnen"
    V -> "schenkte"
""")

parser = nltk.ChartParser(grammar)
for tree in parser.parse(sent.split()):
    tree.pretty_print(unicodelines=True)

                       S                       
     ┌─────────────────┴───────┐                
     │                         VP              
     │                 ┌───────┼────────┐       
     NP                │       NP       NP     
 ┌───┴───────┐         │       │    ┌───┴───┐   
DET          N         V      PRON DET      N  
 │           │         │       │    │       │   
die     Studierende schenkte ihnen ein     Buch



## Aufgabe 4 - Erweiterung Phrasenstrukturgrammatik

### (a) Erweitern Sie die Grammatikregeln aus der vorherigen Aufgabe um lexikalische sowie syntaktische Regeln für: 

1. Eigennamen (können ohne Artikel eine Nominalphrase bilden)
2. PPs als Attribute von NPs

Folgende Sätze sollten nun korrekt analysiert werden:


In [6]:
sents = [
    "die Studierende schenkte ihnen ein Buch",
# neu:
    "Maria schenkte ihnen ein Buch von Noam Chomsky"
]

In [7]:
#Lösung
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> DET N | PRON
    VP -> V NP NP
    DET -> "die" | "ein"
    N -> "Studierende" | "Buch"
    PRON -> "ihnen"
    V -> "schenkte"

    NP -> PROPN | PROPN PROPN
    PROPN -> "Chomsky" | "Noam" | "Maria" | "Moritz"

    PP -> P NP
    P -> "von"
    NP -> NP PP
""")

parser = nltk.ChartParser(grammar)
for sent in sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

                       S                       
     ┌─────────────────┴───────┐                
     │                         VP              
     │                 ┌───────┼────────┐       
     NP                │       NP       NP     
 ┌───┴───────┐         │       │    ┌───┴───┐   
DET          N         V      PRON DET      N  
 │           │         │       │    │       │   
die     Studierende schenkte ihnen ein     Buch

                      S                                
  ┌───────────────────┴───────┐                         
  │                           VP                       
  │      ┌───────┬────────────┴────┐                    
  │      │       │                 NP                  
  │      │       │        ┌────────┴────┐               
  │      │       │        │             PP             
  │      │       │        │        ┌────┴────┐          
  NP     │       NP       NP       │         NP        
  │      │       │    ┌───┴───┐    │    ┌────┴─────┐   

### (b) Stellen Sie außerdem sicher, dass folgende Sätze KEINE Analyse liefern! Passen Sie die Grammatik in (a) ggf. an.



In [8]:
neg_sents = [
    "Studierende schenkte ihnen ein Buch",
    "die Maria schenkte ihnen ein Buch"
]

parser = nltk.ChartParser(grammar)
for sent in neg_sents:
    trees = list(parser.parse(sent.split()))
    if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
    else: print(f"no parse found for: {sent}")

no parse found for: Studierende schenkte ihnen ein Buch
no parse found for: die Maria schenkte ihnen ein Buch


In [9]:
# Hinweis: Analyse möglich wenn statt PROPN N:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> DET N | PRON
    VP -> V NP NP
    DET -> "die" | "ein"
    N -> "Studierende" | "Buch"
    PRON -> "ihnen"
    V -> "schenkte"

    NP -> N | N N
    N -> "Chomsky" | "Noam" | "Maria" | "Moritz"

    PP -> P NP
    P -> "von"
    NP -> NP PP
""")

parser = nltk.ChartParser(grammar)
for sent in neg_sents:
    for tree in parser.parse(sent.split()):
        tree.pretty_print(unicodelines=True)

               S                       
     ┌─────────┴───────┐                
     │                 VP              
     │         ┌───────┼────────┐       
     NP        │       NP       NP     
     │         │       │    ┌───┴───┐   
     N         V      PRON DET      N  
     │         │       │    │       │   
Studierende schenkte ihnen ein     Buch

                 S                       
     ┌───────────┴───────┐                
     │                   VP              
     │           ┌───────┼────────┐       
     NP          │       NP       NP     
 ┌───┴────┐      │       │    ┌───┴───┐   
DET       N      V      PRON DET      N  
 │        │      │       │    │       │   
die     Maria schenkte ihnen ein     Buch

